# Branching ratios
Checking the branching ratios using the Meijer Hamiltonian

## Setting up

In [14]:
import numpy as np
import pickle
import sys
import sympy
from tqdm.notebook import tqdm

sys.path.append('../B-state-classes-and-functions/')

from classes import CoupledBasisState, UncoupledBasisState, State
from functions import matrix_to_states, vector_to_state, find_state_idx_from_state, find_closest_state

from sympy.physics.wigner import wigner_3j

def threej_f(j1,j2,j3,m1,m2,m3):
    return complex(wigner_3j(j1,j2,j3,m1,m2,m3))


with open("B_hamiltonians_symbolic_J1to10_mF0.py",'rb') as f:
    hamiltonians_B = pickle.load(f)
    
with open("TlF_X_state_hamiltonian0to6_2020_03_03.py",'rb') as f:
    hamiltonians_X = pickle.load(f)



def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(x)
        x += dx
    return range_list

#Generate quantum numbers
I_Tl = 1/2             # I1 in Ramsey's notation
I_F  = 1/2             # I2 in Ramsey's notation

#Generate list of ground states
Jmin_g = 0
Jmax_g = 6 # max J value in Hamiltonian

# QN_g = []

# for J in np.arange(Jmin_g, Jmax_g+1):
#     for m1 in np.arange(-I_Tl,I_Tl+1):
#         for m2 in np.arange(-I_F,I_F+1):
#             mJ = 0-m1-m2
#             if np.abs(mJ) <= J:
#                 QN_g.append(UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2,0))
                
QN_g = [UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2,0)
      for J  in ni_range(0, Jmax_g+1)
      for mJ in ni_range(-J,J+1)
      for m1 in ni_range(-I_Tl,I_Tl+1)
      for m2 in ni_range(-I_F,I_F+1)
     ]
                
#Generate list of excited states
Jmin_e = 1
Jmax_e = 10

QN_e = []

Omegas = [-1,1]
for Omega in Omegas:
    for J in np.arange(Jmin_e, Jmax_e+1):
        for m1 in np.arange(-I_Tl,I_Tl+1):
            for m2 in np.arange(-I_F,I_F+1):
                mJ = 0-m1-m2
                if np.abs(mJ) <= J:
                    QN_e.append(UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2,Omega))


### Eigenstates for B

In [15]:
variables = [
    *sympy.symbols('Brot Drot H_const'),
    *sympy.symbols('h1_Tl h1_F'),
    sympy.symbols('q'),
    sympy.symbols('c1'),
    sympy.symbols('c_Tl_p'),
    *sympy.symbols('gL gS')
]

lambdified_hamiltonians_B = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians_B.items()
}

Insert the numerical constants:

In [16]:
#Constants in MHz
Brot = 6687.879e6
Drot = 0.010869e6
H_const = -8.1e-2
h1_Tl = 28789e6
h1_F = 861e6
q = 2.423e6
c1 = -7.83e6
c_Tl_p = 11.17e6
gL = 1
gS = 2

H = {
    H_name : H_fn(
        Brot, Drot, H_const,
        h1_Tl, h1_F,
        q,
        c1,
        c_Tl_p,
        gL, gS
    )
    for H_name, H_fn in lambdified_hamiltonians_B.items()
}



In [17]:
Hff_e = H["Hrot"]+H["H_mhf_Tl"]+H["H_mhf_F"]+H["Hc1"]+H["H_LD"]+H["H_c1_p"]
print(np.allclose(Hff_e, Hff_e.conj().T))
D_e, V_e = np.linalg.eigh(Hff_e)

True


In [18]:
excited_states_all = matrix_to_states(V_e,QN_e)
excited_states = excited_states_all

In [19]:
for state in excited_states[:8]:
    print("")
    state.remove_small_components().transform_to_coupled().print_state(tol=0.01)


-0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

-0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.0142+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.0126+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.0142+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.0126+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

+0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, 

### Eigenstates for X

In [20]:
#Substitute values into hamiltonian
variables = [
    sympy.symbols('Brot'),
    *sympy.symbols('c1 c2 c3 c4'),
    sympy.symbols('D_TlF'),
    *sympy.symbols('mu_J mu_Tl mu_F')
]

lambdified_hamiltonians_X = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians_X.items()
}

#Molecular constants

# Values for rotational constant are from "Microwave Spectral tables: Diatomic molecules" by Lovas & Tiemann (1974). 
# Note that Brot differs from the one given by Ramsey by about 30 MHz.
B_e = 6.689873e9
c1 = 126030.0 
c2 = 17890.0
c3 = 700.0
c4 = -13300.0
alpha = 45.0843e6
Brot = B_e - alpha/2
D_TlF = 4.2282 * 0.393430307 *5.291772e-9/4.135667e-15 # [Hz/(V/cm)]
mu_J = 35 # Hz/G
mu_Tl = 1240.5 #Hz/G
mu_F = 2003.63 #Hz/G

H_g = {
    H_name : H_fn(
        Brot,
        c1, c2, c3, c4,
        D_TlF,
        mu_J, mu_Tl, mu_F
    )
    for H_name, H_fn in lambdified_hamiltonians_X.items()
    }

In [21]:
Hff_g = H_g["Hff"]+0.001*H_g["HZz"]
print(np.allclose(Hff_g, Hff_g.conj().T))
D_g, V_g = np.linalg.eigh(Hff_g)

True


In [22]:
ground_states_all = matrix_to_states(V_g,QN_g)
ground_states = ground_states_all[0:36]

In [23]:
for state in ground_states[:8]:
    print("")
    state.remove_small_components().transform_to_coupled().print_state(tol=0.01)


+1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

-1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 0, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

-0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

-0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2

## Electric dipole matrix elements
Calculate the electric dipole matrix elements between uncoupled basis states in X and B state 

In [24]:
def ED_ME(ground_state, excited_state, pol_vec = np.array([0,0,1]), rme_only = True):
    """
    Function for calculating electric dipole matrix elements between UncoupledBasisStates.
    
    inputs:
    ground_state = UncoupledBasisState object in the X-state of TlF
    excited_state = UncoupledBasisState object in the B-state of TlF
    pol_vec = polarization vector for the light that is driving the transition
    rme_only = True if want to return reduced matrix element, False if want angular component also
    
    returns:
    ME = (reduced) electric dipole matrix element between ground_state and excited_state
    """
    
    #Find quantum numbers for ground state
    J = ground_state.J
    mJ = ground_state.mJ
    I1 = ground_state.I1
    m1 = ground_state.m1
    I2 = ground_state.I2
    m2 = ground_state.m2
    Omega_g = ground_state.Omega
    
    #Find quantum numbers for excited state
    Jp = excited_state.J
    mJp = excited_state.mJ
    I1p = excited_state.I1
    m1p = excited_state.m1
    I2p = excited_state.I2
    m2p = excited_state.m2
    Omega_e = excited_state.Omega
    
    #Calculate the reduced matrix element
    q = Omega_g - Omega_e
    ME = (np.sqrt((2*J+1)*(2*Jp+1))*(-1)**(J-Omega_g)*threej_f(J,1,Jp,-Omega_g, q, Omega_e) *int(np.abs(q) <= 1)
          * int(I1 == I1p and m1 == m1p and I2 == I2p and m2 == m2p))
    
    #If we want the complete matrix element, calculate angular part
    if not rme_only:
        
        #Calculate elements of the polarization vector in spherical basis
        p_vec = {}
        p_vec[-1] = -1/np.sqrt(2) * (pol_vec[0] + 1j *pol_vec[1])
        p_vec[0] = pol_vec[2]
        p_vec[1] = +1/np.sqrt(2) * (pol_vec[0] - 1j *pol_vec[1])
        
        #Calculate the value of p that connects the states
        p = (mJ-mJp) 
        p = p*int(np.abs(p) <= 1)
        #Multiply RME by the angular part
        ME = ME * (-1)**(J-mJ) * threej_f(J,1,Jp, -mJ, p, mJp) * int(np.abs(p) <= 1)
    
    #Return the matrix element
    return ME
    
    
    

In [25]:
#Generate a matrix that contains the reduced electric dipole matrix elements between all excited and ground states of interest             
def generate_ME_matrix(H, QN_g=QN_g, QN_e = QN_e):
    #result = sympy.zeros(len(ground_states),len(excited_states))
    result = np.zeros((len(QN_g),len(QN_e)),dtype = complex)
    for i,a in tqdm(enumerate(QN_g)):
        for j,b in enumerate(QN_e):
            result[i,j] = H(a,b,rme_only=False)#/np.sqrt((2*a.J+1))
            
    return result

MEs = generate_ME_matrix(ED_ME)
MEs_states = V_g.conj().T @ MEs @ V_e

def calculate_branching_ratio(ground_state, excited_state, MEs = MEs, QN_g = QN_g, 
                              QN_e = QN_e):
    
    #Get the ground_state and excited_state state vectors
    vec_g = ground_state.state_vector(QN_g)
    vec_e = excited_state.state_vector(QN_e)
    
    #Find the matrix element between ground_state and excited_state (for calculating partial width to ground_state)
    ME_ge = vec_g.conj().T @ MEs @ vec_e
    
    #Find the matrix elements between the excited_state and all possible ground states
    #so the total width of the excited state can be calculated
    ME_e_all = V_g.conj().T @ MEs @ vec_e
    
    #Calculate branching ratio
    branching_ratio = ME_ge*ME_ge.conj()/(ME_e_all.conj().T @ ME_e_all)
    
    return branching_ratio
    


In [13]:
#Loop over all excited states and calculate their branching ratios to all ground states
for excited_state in tqdm(excited_states):
    print(10*"*")
    print("|e> =")
    excited_state.remove_small_components().transform_to_coupled().print_state()
    print("\nBranching ratios:")
    #Loop over ground states
    for ground_state in ground_states:
        br = np.real(calculate_branching_ratio(ground_state, excited_state))
        if br > 0.0001:
            print("{:5f} to \n|g> =".format(br))
            ground_state.remove_small_components().transform_to_coupled().print_state(tol = 0.01)
            print("")
        

**********
|e> =
-0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.249085 to 
|g> =
-0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.249085 to 
|g> =
+0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.249084 to 
|g> =
-0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.084249 to 
|g> =
-0.0886+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9961+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = -1, I

KeyboardInterrupt: 

In [ ]:
def branching_ratio_table(excited_state, ground_states):
    nominal_state = excited_state.find_largest_component()
    nominal_J = nominal_state.J
    I1 = 1/2
    I2 = 1/2
    print(10*"*")
    print("|e> =")
    excited_state.remove_small_components().transform_to_coupled().print_state()
    print("\nBranching ratios:")
    
    branching_ratios = {-3:0, -2:0,-1:0, 0:0, 1:0, 2:0, 3:0}
    
    for ground_state in ground_states:
        J_g = ground_state.find_largest_component().J
        DeltaJ = J_g-nominal_J
        
        if np.abs(DeltaJ) < 4:
            br = np.real(calculate_branching_ratio(ground_state, excited_state))
            branching_ratios[DeltaJ] += br
            
    for DeltaJ, BR in branching_ratios.items():
        print("{:d}: {:.5f}".format(DeltaJ, BR))

In [ ]:
for excited_state in tqdm(excited_states):
    branching_ratio_table(excited_state,ground_states)

In [ ]:
excited_state_approx = 1/np.sqrt(2)*(CoupledBasisState(J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = +1)
                                     + CoupledBasisState(J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1))
excited_state = find_closest_state(Hff_e,excited_state_approx,QN_e)
print("|e> =")
excited_state.remove_small_components().transform_to_coupled().print_state()
print("\nBranching ratios:")
#Loop over ground states
for ground_state in ground_states:
    br = np.real(calculate_branching_ratio(ground_state, excited_state))
    if br > 0.0001:
        print("{:5f} to \n|g> =".format(br))
        ground_state.remove_small_components().transform_to_coupled().print_state(tol = 0.01)
        print("")